In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [2]:
tree_model=tree.DecisionTreeClassifier(max_depth=8)

In [3]:
bank=pd.read_excel("Bank_Personal_Loan_Modelling.xlsx",sheet_name='Data')

In [4]:
bank.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [5]:
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)

In [6]:
bank.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [7]:
features=pd.DataFrame([bank["Age"],bank["Experience"],bank["Income"],bank["Family"],bank["CCAvg"],bank["Education"],bank["Mortgage"],bank["Securities Account"],bank["CD Account"],bank["Online"],bank["CreditCard"]]).T

In [8]:
rf_model.fit(X=features,y=bank["Personal Loan"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
print("OOB Accuracy")
print(rf_model.oob_score_);

OOB Accuracy
0.988


In [10]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)

Age 0.04479989270334714
Experience 0.044331528014268706
Income 0.3415920721263146
Family 0.0977579096696056
CCAvg 0.18128449674792033
Education 0.16834502268600873
Mortgage 0.0442914284897023
Securities Account 0.005455531374463444
CD Account 0.05392190584458907
Online 0.008337753708305164
CreditCard 0.00988245863547494


In [11]:
bank.isnull().isnull().sum().sum()

0

In [12]:
predictors=pd.DataFrame([bank["Income"],bank["CCAvg"],bank["Education"],bank["Family"]]).T

In [13]:
tree_model.fit(X=predictors,y=bank["Personal Loan"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [14]:
with open("Bank_Modelling.dot",'w') as f:
    f=tree.export_graphviz(tree_model,feature_names=["Income","CCAvg","Education","Family"],out_file=f)


In [15]:
tree_model.score(X=predictors,y=bank["Personal Loan"])

0.9954

In [16]:
predictors1=pd.DataFrame([bank["Income"],bank["CCAvg"],bank["Education"]]).T

In [17]:
tree_model.fit(X=predictors1,y=bank["Personal Loan"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [18]:
with open("Bank_ModellingWithoutFamily.dot",'w') as f:
    f=tree.export_graphviz(tree_model,feature_names=["Income","CCAvg","Education"],out_file=f)

In [19]:
tree_model.score(X=predictors1,y=bank["Personal Loan"])

0.9816

### The first Model has 99.54% Accuracy hence we can use that Bank Modeeling Model to decide if someone is eligible for a personal loan or not

In [20]:
from sklearn.tree import export_text

In [21]:
feature=["Income","CCAvg","Education"]

In [22]:
r=export_text(tree_model,feature_names=feature)
print(r)

|--- Income <= 113.50
|   |--- CCAvg <= 2.95
|   |   |--- Income <= 106.50
|   |   |   |--- class: 0
|   |   |--- Income >  106.50
|   |   |   |--- Education <= 1.50
|   |   |   |   |--- CCAvg <= 0.35
|   |   |   |   |   |--- CCAvg <= 0.25
|   |   |   |   |   |   |--- Income <= 112.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Income >  112.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- CCAvg >  0.25
|   |   |   |   |   |   |--- Income <= 111.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- Income >  111.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- CCAvg >  0.35
|   |   |   |   |   |--- Income <= 109.50
|   |   |   |   |   |   |--- CCAvg <= 1.75
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- CCAvg >  1.75
|   |   |   |   |   |   |   |--- CCAvg <= 1.90
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- CCAvg >  1.90
|   |   |   |   |   |   